In [ ]:
%load_ext autoreload
%autoreload 2

**In case** there is issue with reloading, run the following to delete compiled Python cache.

This is **typically** not needed.
Often, restarting the notebook works better.

In [ ]:
!cd ..; cd arguments; rm -rf __pycache__
!cd ..; cd constants; rm -rf __pycache__
!cd ..; cd data; rm -rf __pycache__
!cd ..; cd logger; rm -rf __pycache__
!cd ..; cd models; rm -rf __pycache__
!cd ..; cd utils; rm -rf __pycache__

# Entrace Notebook for Forex Trading

This is for testing only. For batched runs a main file is still required.

Keep autoreload for faster Python module reloading.

## Imports

Import Python default packges first, then those requiring `pip`, finally modules in the code structure. Keep alphabetical order.

In [1]:
import sys
sys.path.append('..')

import os

import pandas as pd
import tqdm

from arguments import *
from constants import *
from data import *
import models as supported_models
from training_tools import *
from utils import *

## Preparations

* Setup interface with Google Cloud bucket.
* Pretend to have command line argument.
* Prepare data
* Define models

In [2]:
storage = GCStorage.get_CloudFS(project_name=PROJECT_NAME,
                                bucket_name=GC_BUCKET,
                                credential_path=CREDENTIAL_PATH)

Unique instance for GCStorage has been created


In [3]:
all_args = parse_from_string('--exp_name=jupyter --device=cpu --num_candles=4 --num_workers=1 --fast_debug=True --candle_interval=30')

Starting new experiment at 2020-05-04 07:27:55
User: jingboyang
Host: pg-cpu-1
{'misc_args': {'exp_name': 'jingboyang_jupyter_7', 'log_level': 0, 'fast_debug': True, 'save_dir': 'experiments/2020-05-04/jingboyang_jupyter_7', 'log_file': 'experiments/2020-05-04/jingboyang_jupyter_7/run_log.txt'}, 'data_args': {'candle_interval': 30, 'num_candles': 4, 'num_iterval_ahead': 4, 'currency_pair': 'USDCAD', 'num_workers': 1}, 'train_args': {'device': 'cpu', 'disp_steps': 1, 'max_epochs': 100, 'batch_size': 256, 'learning_rate': 0.0001, 'weight_decay': 0.9, 'clipping_value': 1.0}, 'model_args': {'model_type': 'DummyModel', 'emb_size': 32, 'hidden_size': 64, 'num_layers': 3}}
Step 0/False Texts
	[setup/command_line: /opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py --exp_name=jupyter --device=cpu --num_candles=4 --num_workers=1 --fast_debug=True --candle_interval=30]
	[setup/arguments: {   'data_args': {   'candle_interval': 30,
                     'currency_pair': 'USDCAD',
    

In [4]:
logger = all_args.misc_args.logger
device = all_args.train_args.device

In [5]:
train_loader, valid_loader = get_dataloaders(all_args)

In [ ]:
model_init_func = supported_models.__dict__[all_args.model_args.model_type]
model = model_init_func(all_args.model_args) # TODO: JBY. Enable Dataparallel, if needed.

## Define Training Tools

In [ ]:
loss_func = get_loss()# TODO (JBY): Replace with proper evaluator
optimizer = Optimizer(all_args, model.parameters(), len(train_loader.dataset))

## Train (Deep Learning)

At the moment we only have a validation set, a test set has not been created yet.

In [ ]:
for epoch in range(all_args.train_args.max_epochs):
    for batch in tqdm(train_loader):
        
        optimizer.zero_grad()
        time_series, ground_truth = batch

        pred = model(input_seq)
        
        loss = loss_func(output, target_seq.view(-1).long())
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly

    print(f'Epoch: {epoch + 1}/{all_args.train_args.max_epochs}')
    print(f'Loss: {loss.item()}')

## Train (Machine Learning)

Here we only use the tools to gather data :P

In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor

In [ ]:
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])

In [9]:
train_x = []
train_y = []
for batch in tqdm(train_loader):    
    time_series, ground_truth = batch
    
    print(time_series.shape)
    print(ground_truth)
    train_x.append(time_series.numpy().flatten())
    train_y.append(ground_truth.numpy().flatten()[0])



  0%|          | 0/56 [00:00<?, ?it/s]

Index[11690] (2019-02-08 01:25:00, 2019-02-08 01:27:00)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f57d8e5e9e0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Index[12378] (2019-02-08 07:09:00, 2019-02-08 07:11:00)
Index[5987] (2019-02-06 01:53:30, 2019-02-06 01:55:30)
Index[3946] (2019-02-05 08:53:00, 2019-02-05 08:55:00)
Index[12862] (2019-02-08 11:11:00, 2019-02-08 11:13:00)
Index[62] (2019-02-04 00:31:00, 2019-02-04 00:33:00)
Index[13621] (2019-02-08 17:30:30, 2019-02-08 17:32:30)
Index[1356] (2019-02-04 11:18:00, 2019-02-04 11:20:00)
LP 4 has insufficient number of candles 1
Index[4604] (2019-02-05 14:22:00, 2019-02-05 14:24:00)
Index[9711] (2019-02-07 08:55:30, 2019-02-07 08:57:30)
Index[758] (2019-02-04 06:19:00, 2019-02-04 06:21:00)
Index[6978] (2019-02-06 10:09:00, 2019-02-06 10:11:00)
Index[5436] (2019-02-05 21:18:00, 2019-02-05 21:20:00)
LP 1 has insufficient number of candles 1
Index[3858] (2019-02-05 08:09:00, 2019-02-05 08:11:00)
Index[3695] (2019-02-05 06:47:30, 2019-02-05 06:49:30)
LP 1 has insufficient points
Index[10142] (2019-02-07 12:31:00, 2019-02-07 12:33:00)
Index[2328] (2019-02-04 19:24:00, 2019-02-04 19:26:00)
Index[

AssertionError: Caught AssertionError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "../data/data_manager.py", line 90, in __getitem__
    get_df=False)
  File "../utils/format_conversion.py", line 116, in generate_candles
    assert start_i is not None
AssertionError


Index[1159] (2019-02-04 09:39:30, 2019-02-04 09:41:30)
Index[5671] (2019-02-05 23:15:30, 2019-02-05 23:17:30)
LP 4 has insufficient points
Index[950] (2019-02-04 07:55:00, 2019-02-04 07:57:00)
Index[8692] (2019-02-07 00:26:00, 2019-02-07 00:28:00)
LP 1 has insufficient number of candles 1
Index[8267] (2019-02-06 20:53:30, 2019-02-06 20:55:30)
Index[12056] (2019-02-08 04:28:00, 2019-02-08 04:30:00)
Index[10793] (2019-02-07 17:56:30, 2019-02-07 17:58:30)
Index[6595] (2019-02-06 06:57:30, 2019-02-06 06:59:30)
Index[6895] (2019-02-06 09:27:30, 2019-02-06 09:29:30)
Index[7065] (2019-02-06 10:52:30, 2019-02-06 10:54:30)
LP 1 has insufficient number of candles 0
Index[3694] (2019-02-05 06:47:00, 2019-02-05 06:49:00)
LP 1 has insufficient points
Index[13540] (2019-02-08 16:50:00, 2019-02-08 16:52:00)
Index[1347] (2019-02-04 11:13:30, 2019-02-04 11:15:30)
Index[7199] (2019-02-06 11:59:30, 2019-02-06 12:01:30)
Index[13158] (2019-02-08 13:39:00, 2019-02-08 13:41:00)
Index[1207] (2019-02-04 10:03:

In [ ]:
ereg = ereg.fit(X, y)

In [10]:
train_x

[]